In [7]:
import joblib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Install xlrd package
%pip install xlrd

all_df = pd.read_excel('FinalTestDataset2024.xls', index_col=False)
IDs = all_df['ID']
all_df.drop('ID', axis=1, inplace=True)
all_df.head()

saved_model = joblib.load('pcr_classification_model.joblib')
model = saved_model['model']
scaler = saved_model['scaler']
lda = saved_model['lda']
selected_features_indices = saved_model['selected_features_indices']

Note: you may need to restart the kernel to use updated packages.


### Pre-Process testing data

In [8]:
from sklearn.impute import SimpleImputer

# Replace missing values with median of the column
imputer = SimpleImputer(strategy="median", missing_values=999)
all_df[:] = imputer.fit_transform(all_df)

# Standardize the data
Xs = scaler.transform(all_df)

### Apply Feature Selection and Dimensionality Reduction training strategy

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Select required features (ER, HER2 and Gene)
non_mri_features = Xs[:, selected_features_indices]

# Select MRI features
mri_indices = list(range(11, Xs.shape[1]))
mri = Xs[:, mri_indices]

# Apply LDA to MRI features
Xs_lda = lda.transform(Xs[:, 11:])

# Combine required features with LDA transformed features
Xs = np.hstack((non_mri_features, Xs_lda))

### Predict Classification

In [10]:
result = model.predict(Xs)
print(result)

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0
 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 1 1 1
 0 0 0 1 1 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0]


# Part 2: RFS Regression

### Pre-process for Regression

In [11]:
### Save to results to csv
clf_outcome = pd.DataFrame({'ID': IDs,'pCR (outcome)': result,})
clf_outcome.to_csv('PCRPrediction.csv', index=False)